In [1]:
# TESTLIB_PATH = "/data/Alchemist-TestLib/target/testlib.dylib"
TESTLIB_PATH = "/Users/kai/Projects/AlLib/target/testlib.dylib"
# TESTLIB_PATH = "/usr/local/TestLib/target/testlib.so"
HOSTNAME = "localhost"
PORT = 24960

In [2]:
from alchemist import *
import numpy as np

als = AlchemistSession(worker_buffer_length=10000000)        # Start Alchemist session

als.connect_to_alchemist(HOSTNAME, PORT)

Starting Alchemist session ... ready
Connecting to Alchemist at localhost:24960 ...  
          Client ID:                  0
          Session ID:                 0
          Command code:               1 (HANDSHAKE)
          Error code:                 0 (NONE)
          Message body length:        181
         -------------------------------------------------------------------------------
          BYTE                        4
          SHORT                       1234
          STRING                      ABCD
          DOUBLE                      1.11
          DOUBLE                      2.22
          MATRIX BLOCK                4x3 | [0 1 2 3] | [0 1 2]
          INT                         10010
 
          Client ID:                  1
          Session ID:                 1
          Command code:               1 (HANDSHAKE)
          Error code:                 0 (NONE)
          Message body length:        19
         -----------------------------------------------------

In [3]:
als.request_workers(2) 

Requesting 2 workers from Alchemist
 
          Client ID:                  1
          Session ID:                 1
          Command code:               11 (REQUEST_WORKERS)
          Error code:                 0 (NONE)
          Message body length:        3
         -------------------------------------------------------------------------------
          SHORT                       2
 
          Client ID:                  1
          Session ID:                 1
          Command code:               11 (REQUEST_WORKERS)
          Error code:                 0 (NONE)
          Message body length:        79
         -------------------------------------------------------------------------------
          SHORT                       2
          WORKER INFO                 
                                      ID:           1
                                      Hostname:     KaisMacBookPro.local
                                      Address:      0.0.0.0
                       

In [4]:
TestLib = als.load_library("TestLib", TESTLIB_PATH)

 
          Client ID:                  1
          Session ID:                 1
          Command code:               22 (LOAD_LIBRARY)
          Error code:                 0 (NONE)
          Message body length:        59
         -------------------------------------------------------------------------------
          STRING                      TestLib
          STRING                      /Users/kai/Projects/AlLib/target/testlib.dylib
 
          Client ID:                  1
          Session ID:                 1
          Command code:               22 (LOAD_LIBRARY)
          Error code:                 0 (NONE)
          Message body length:        2
         -------------------------------------------------------------------------------
          LIBRARY ID                  3
Library 'TestLib' successfully loaded.


In [5]:
A = np.random.rand(2000,1000)

A_handle = als.send_matrix(matrix=A, print_times=True, layout="VC_STAR")

Sending array info to Alchemist ...  
          Client ID:                  1
          Session ID:                 1
          Command code:               31 (SEND_MATRIX_INFO)
          Error code:                 0 (NONE)
          Message body length:        25
         -------------------------------------------------------------------------------
          STRING                      
          LONG                        2000
          LONG                        1000
          BYTE                        0
          BYTE                        11
 
          Client ID:                  1
          Session ID:                 1
          Command code:               31 (SEND_MATRIX_INFO)
          Error code:                 0 (NONE)
          Message body length:        39
         -------------------------------------------------------------------------------
          MATRIX INFO                 
                                      Name:                  
                   

 
          Client ID:                  1
          Session ID:                 2
          Command code:               34 (SEND_MATRIX_BLOCKS)
          Error code:                 0 (NONE)
          Message body length:        8
         -------------------------------------------------------------------------------
          SHORT                       1
          INT                         1
 
          Client ID:                  1
          Session ID:                 2
          Command code:               34 (SEND_MATRIX_BLOCKS)
          Error code:                 0 (NONE)
          Message body length:        8000053
         -------------------------------------------------------------------------------
          MATRIX ID                   1
          MATRIX BLOCK                1000x1000 | [   1    3    5    7    9   11   13   15   17   19   21   23   25   27
   29   31   33   35   37   39   41   43   45   47   49   51   53   55
   57   59   61   63   65   67   69   71  

 
          Client ID:                  1
          Session ID:                 2
          Command code:               34 (SEND_MATRIX_BLOCKS)
          Error code:                 0 (NONE)
          Message body length:        8
         -------------------------------------------------------------------------------
          SHORT                       1
          INT                         1
done (6.4103e-01s)

Data transfer times breakdown
  ---------------------------------------------------------------------------------------------------------------
    Worker  |   Serialization time   |       Send time        |      Receive time      |  Deserialization time  
  ---------------------------------------------------------------------------------------------------------------
        1   |       2.4099e-02       |       5.0671e-03       |       3.6716e-05       |       2.8610e-06       
   - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

In [6]:
AA = als.fetch_matrix(A_handle, print_times=True)

Fetching data for array  from Alchemist ...  
          Client ID:                  1
          Session ID:                 2
          Command code:               36 (REQUEST_MATRIX_BLOCKS)
          Error code:                 0 (NONE)
          Message body length:        57
         -------------------------------------------------------------------------------
          MATRIX ID                   1
          LONG                        0
          LONG                        1998
          LONG                        2
          LONG                        0
          LONG                        999
          LONG                        1


TypeError: cannot unpack non-iterable int object

In [ ]:
import matplotlib.pyplot as plt

ax = plt.imshow(A[1900:2000, :] - AA[1900:2000, :])
plt.colorbar(ax)
plt.show()

print(np.linalg.norm(A - AA))

In [ ]:
in_byte = 9
in_char = 'y'
in_short = 9876
in_int = 987654321
in_long = 98765432123456789
in_float = 77.77777777
in_double = 88.88888888888888888
in_string = "test string"

TestLib.greet(in_byte, in_char, in_short, in_int, in_long, in_float, in_double, in_string)

In [ ]:
rank = 20
S_handle, U_handle, V_handle = TestLib.truncated_svd(A_handle, rank)

In [ ]:
S = als.fetch_matrix(S_handle, print_times=True)

print(np.flipud(S).transpose())

In [ ]:
V = als.fetch_matrix(V_handle, print_times=True)
U = als.fetch_matrix(U_handle, print_times=True)
S = als.fetch_matrix(S_handle, print_times=True)

In [ ]:
u, s, vh = np.linalg.svd(A, full_matrices=True)

In [ ]:
print(np.linalg.norm(abs(V.T) - abs(np.flipud(vh[0:rank,:]))))
print(np.linalg.norm(abs(U) - abs(np.fliplr(u[:,0:rank]))))
print(np.linalg.norm(np.flip(S).T - s[0:rank]))

In [ ]:
als.stop()

In [ ]:
print(s)